# Prediction using NN

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
import os

## Load Dataset

In [ ]:
data_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data'

file_path = os.path.join(data_folder_path, 'train.csv')
train_df = pd.read_csv(file_path)

file_path = os.path.join(data_folder_path, 'test.csv')
test_df = pd.read_csv(file_path)

train_df.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


## EDA & Cleaning

In [ ]:
# Drop the columns that are not useful for the model - PassengerId, Name, Ticket
train_df = train_df.drop(['PassengerId','Name','Ticket'], axis=1)

train_df.head(10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked
0,0,3,male,22.0,1,0,7.2500,NaN,S
1,1,1,female,38.0,1,0,71.2833,C85,C
2,1,3,female,26.0,0,0,7.9250,NaN,S
3,1,1,female,35.0,1,0,53.1000,C123,S
4,0,3,male,35.0,0,0,8.0500,NaN,S
5,0,3,male,NaN,0,0,8.4583,NaN,Q
6,0,1,male,54.0,0,0,51.8625,E46,S
7,0,3,male,2.0,3,1,21.0750,NaN,S
8,1,3,female,27.0,0,2,11.1333,NaN,S
9,1,2,female,14.0,1,0,30.0708,NaN,C


In [ ]:
# check for missing values
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
train_df.shape

(891, 9)

In [ ]:
# drop empty embarked rows
train_df.dropna(subset=['Embarked'])

# fill missing age values with the mean
train_df['Age'] = train_df['Age'].fillna(train_df['Age'].mean())

# check for missing values
train_df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age           0
SibSp         0
Parch         0
Fare          0
Cabin       687
Embarked      2
dtype: int64

In [ ]:
# Transform the categorical data into numerical data
train_df = pd.get_dummies(train_df, columns=['Sex','Embarked'], dtype='int64')

train_df.head(10)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,NaN,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,C85,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,NaN,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,C123,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,NaN,0,1,0,0,1
5,0,3,29.699118,0,0,8.4583,NaN,0,1,0,1,0
6,0,1,54.000000,0,0,51.8625,E46,0,1,0,0,1
7,0,3,2.000000,3,1,21.0750,NaN,0,1,0,0,1
8,1,3,27.000000,0,2,11.1333,NaN,1,0,0,0,1
9,1,2,14.000000,1,0,30.0708,NaN,1,0,1,0,0


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'Cabin' is a categorical column in your DataFrame 'train_df'
cabin_encoder = LabelEncoder()
train_df['Cabin'] = cabin_encoder.fit_transform(train_df['Cabin'].astype(str))

# Now, 'Cabin' is transformed into numerical values
train_df.head(10)

,Survived,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,0,3,22.000000,1,0,7.2500,147,0,1,0,0,1
1,1,1,38.000000,1,0,71.2833,81,1,0,1,0,0
2,1,3,26.000000,0,0,7.9250,147,1,0,0,0,1
3,1,1,35.000000,1,0,53.1000,55,1,0,0,0,1
4,0,3,35.000000,0,0,8.0500,147,0,1,0,0,1
5,0,3,29.699118,0,0,8.4583,147,0,1,0,1,0
6,0,1,54.000000,0,0,51.8625,129,0,1,0,0,1
7,0,3,2.000000,3,1,21.0750,147,0,1,0,0,1
8,1,3,27.000000,0,2,11.1333,147,1,0,0,0,1
9,1,2,14.000000,1,0,30.0708,147,1,0,1,0,0


In [ ]:
train_df.shape

(891, 12)

## Model training

In [ ]:
train_data = train_df.drop('Survived', axis=1)
train_labels = train_df['Survived']

In [ ]:
train_data.head()

,Pclass,Age,SibSp,Parch,Fare,Cabin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S
0,3,22.0,1,0,7.2500,147,0,1,0,0,1
1,1,38.0,1,0,71.2833,81,1,0,1,0,0
2,3,26.0,0,0,7.9250,147,1,0,0,0,1
3,1,35.0,1,0,53.1000,55,1,0,0,0,1
4,3,35.0,0,0,8.0500,147,0,1,0,0,1


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_data, train_labels))

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(11,)),  # input layer (1)
    tf.keras.layers.Dense(256, activation='tanh'),  # hidden layer (2)
    tf.keras.layers.Dense(128, activation='tanh'),
    tf.keras.layers.Dense(10, activation='sigmoid') # output layer (3)
])

model.compile(optimizer='nadam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, train_labels, epochs=100, batch_size=10)

Epoch 1/100
90/90 [==============================] - 2s 2ms/step - loss: 0.7013 - accuracy: 0.6364
Epoch 2/100
90/90 [==============================] - 0s 2ms/step - loss: 0.6048 - accuracy: 0.6902
Epoch 3/100
90/90 [==============================] - 0s 2ms/step - loss: 0.5744 - accuracy: 0.7003
Epoch 4/100
90/90 [==============================] - 0s 2ms/step - loss: 0.5570 - accuracy: 0.7194
Epoch 5/100
90/90 [==============================] - 0s 2ms/step - loss: 0.5226 - accuracy: 0.7587
Epoch 6/100
90/90 [==============================] - 0s 2ms/step - loss: 0.4877 - accuracy: 0.7654
Epoch 7/100
90/90 [==============================] - 0s 2ms/step - loss: 0.4773 - accuracy: 0.7722
Epoch 8/100
90/90 [==============================] - 0s 2ms/step - loss: 0.4877 - accuracy: 0.7710
Epoch 9/100
90/90 [==============================] - 0s 2ms/step - loss: 0.4478 - accuracy: 0.8036
Epoch 10/100
90/90 [==============================] - 0s 2ms/step - loss: 0.4663 - accuracy: 0.7890
Epoch 11/

In [ ]:
# evaluate model
model.evaluate(train_data, train_labels)

28/28 [==============================] - 0s 3ms/step - loss: 0.3345 - accuracy: 0.8676


[0.33450770378112793, 0.8675645589828491]

## Model Prediction

In [ ]:
test_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
def prediction_to_kaggle_format(model, threshold=0.5):
    proba_survive = model.predict(test_data, verbose=0)[:,0]
    return pd.DataFrame({
        "PassengerId": test_df["PassengerId"],
        "Survived": (proba_survive >= threshold).astype(int)
    })

def make_submission(kaggle_predictions):
    path = os.path.join(data_folder_path, 'titanic_submission.csv')
    # path="data/tf_lead_submission.csv"
    kaggle_predictions.to_csv(path, index=False)
    print(f"Submission exported to {path}")

kaggle_predictions = prediction_to_kaggle_format(model)
make_submission(kaggle_predictions)
# !head /content/drive/MyDrive/Colab Notebooks/Data/titanic_submission.csv

Submission exported to /content/drive/MyDrive/Colab Notebooks/Data/titanic_submission.csv
